<a href="https://colab.research.google.com/github/navneetkrc/langchain_colab_experiments/blob/main/gpt_cache_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat

[This example](https://gptcache.readthedocs.io/en/latest/bootcamp/openai/chat.html) will show you how to chat with GPT, the original example is on [OpenAI Example](https://platform.openai.com/docs/guides/chat/introduction), the difference is that we will teach you how to cache the  response for exact and similar matches with **gptcache**, it will be very simple, you just need to add an extra step to initialize the cache.


In [ ]:
!pip install -q gptcache
!pip install openai

Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems.

> We can run `os.environ` to set the environment variable in colab.

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

Then we can learn the usage and acceleration effect of gptcache by the following code, which consists of three parts, the original openai way, the exact search and the similar search.


## OpenAI API original usage

In [7]:
import time
import openai


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']


question = 'what‘s github?'

# OpenAI API original usage
start_time = time.time()
response = openai.ChatCompletion.create(
  model='gpt-3.5-turbo',
  messages=[
    {
        'role': 'user',
        'content': question
    }
  ],
)
print(f'Question: {question}')
print("Time consuming: {:.2f}s".format(time.time() - start_time))
print(f'Answer: {response_text(response)}\n')

Question: what‘s github?
Time consuming: 6.96s
Answer: GitHub is a web-based platform that allows developers to collaborate on software development projects. It is a repository hosting service which provides version control, issue tracking, collaborative coding, and other features for free or at a cost. GitHub provides a platform for developers to store and manage their code, to track and manage changes to code, and to collaborate on coding projects with other developers. It is widely used in the open-source community and by companies for internal development projects. GitHub is also home to a vast library of open-source software that anyone can download, use, and contribute to.



## OpenAI API + GPTCache, exact match cache

Initalize the cache to run GPTCache and import `openai` form `gptcache.adapter`, which will automatically set the map data manager to match the exact cahe, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

And if you ask ChatGPT the exact same two questions, the answer to the second question will be obtained from the cache without requesting ChatGPT again.

In [8]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

print("Cache loading.....")

# To use GPTCache, that's all you need
# -------------------------------------------------
from gptcache import cache
from gptcache.adapter import openai

cache.init()
cache.set_openai_key()
# -------------------------------------------------

question = "what's github"
for _ in range(2):
    start_time = time.time()
    response = openai.ChatCompletion.create(
      model='gpt-3.5-turbo',
      messages=[
        {
            'role': 'user',
            'content': question
        }
      ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 5.06s
Answer: GitHub is a web-based platform that is designed for hosting and collaborating on software development projects using version control. It allows users to store their code in repositories and collaborate with other developers on projects. GitHub also offers a variety of tools and services for project management, like issue tracking, code review, and continuous integration. It is used by developers worldwide for open source and private projects, whether as a team or an individual developer.

Question: what's github
Time consuming: 6.25s
Answer: GitHub is a web-based platform that is designed for hosting and collaborating on software development projects using version control. It allows users to store their code in repositories and collaborate with other developers on projects. GitHub also offers a variety of tools and services for project management, like issue tracking, code review, and continuous integration. It is

## OpenAI API + GPTCache, similar search cache

Set the cache with `embedding_func` to generate embedding for the text, and `data_manager` to manager the cache data, `similarity_evaluation` to evaluate the similarities, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

After obtaining an answer from ChatGPT in response to several similar questions, the answers to subsequent questions can be retrieved from the cache without the need to request ChatGPT again.

In [10]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

from gptcache import cache
from gptcache.adapter import openai
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation

print("Cache loading.....")

onnx = Onnx()
data_manager = get_data_manager(CacheBase("sqlite"), VectorBase("faiss", dimension=onnx.dimension))
cache.init(
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

questions = [
    "what's github",
    "can you explain what GitHub is",
    "can you tell me more about GitHub",
    "what is the purpose of GitHub"
]

for question in questions:
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': question
            }
        ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 8.48s
Answer: GitHub is a platform for software developers to collaborate, share, and work on open-source code. It provides a web-based interface for version control and facilitates collaboration among developers. Using GitHub, developers can share and contribute to open-source projects, track issues, and merge code changes from multiple contributors. Additionally, GitHub provides a range of tools for project management, documentation, and code review. It has become one of the most popular platforms for open-source development due to its ease of use and extensive community support.

Question: can you explain what GitHub is
Time consuming: 0.75s
Answer: GitHub is a platform for software developers to collaborate, share, and work on open-source code. It provides a web-based interface for version control and facilitates collaboration among developers. Using GitHub, developers can share and contribute to open-source projects, track